<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Send data to spreadsheet
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Google%20Sheets/Google_Sheets_Send_data.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Google+Sheets+-+Send+data:+Error+short+description">Bug report</a>

**Tags:** #googlesheets #gsheet #data #naas_drivers #operations #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-04-12 (Created: 2022-03-04)

**Description:** This notebook allows to send data to Google Sheets to a Google Sheets spreadsheet.

## Input

### Import libraries

In [ ]:
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date
import naas_data_product

### Setup variables
**Inputs**
- `file_name`: Name of the file to be retrieved.
- `input_dir`: Input directory to retrieve file from.

**Outputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `append`: If False, data will be canceled and replaced.

In [ ]:
# Inputs
file_name = "linkedin_posts"
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "content-engine", date.today().isoformat())

# Outputs
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1GDVJxBajYx7VR48yKGnSvMsEjzXvqI94hlmPXI5qEp8/edit#gid=0" or naas.secret.get("MY_AI_AVATAR_SPREADSHEET")
sheet_name = "CONTENT"
append = False

## Model

### Get data from Google Sheets spreadsheet

In [ ]:
df_gsheet = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
print("Rows:", len(df_gsheet))
df_gsheet.head(1)

### Get posts from local

In [ ]:
df_posts = pload(input_dir, file_name)    
print("Rows:", len(df_posts))
df_posts.head(1)

### Cleaning data

In [ ]:
df = df_posts.copy()

# Select
to_select = [
    "AUTHOR_NAME",
    "PUBLISHED_DATE",
    "TITLE",
    "TEXT",
    "CHARACTER_COUNT",
    "TAGS",
    "VIEWS",
    "LIKES",
    "COMMENTS",
    "SHARES",
    "ENGAGEMENT_SCORE",
    "POST_URL"
]

to_rename = {
    "POST_URL": "CONTENT_URL",
    "AUTHOR_NAME": "ENTITY",
    "TEXT": "CONTENT",
    "CHARACTER_COUNT": "CONTENT_LENGTH",
    "TAGS": "KEYWORDS",
}
df = df[to_select]
df = df.rename(columns=to_rename)
df.insert(loc=1, column="SCENARIO", value=date.today().isoformat())
df.insert(loc=2, column="SOURCE", value="LinkedIn")
df.insert(loc=4, column="DATE", value=pd.to_datetime(df['PUBLISHED_DATE'].str[:19], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d'))
df.insert(loc=5, column="TIME", value=pd.to_datetime(df['PUBLISHED_DATE'].str[:19], format='%Y-%m-%d %H:%M:%S').dt.strftime('%H:%M:%S'))

# Drop duplicates
df = pd.concat([df, df_gsheet])
df = df.drop_duplicates("CONTENT_URL", keep='first').reset_index(drop=True)
print("Rows:", len(df))
df.head(11)

## Output

### Send data to Google Sheets spreadsheet

In [ ]:
gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name, data=df, append=append)